In [216]:
import pandas as pd
import numpy as np
import operator

In [217]:
submission_file_list = ['submissions/PLSR_100_BOW_all_pca_2048_tags_80_cos_0.26727.csv', 'submissions/0.352_plsr_800_pool5_to_bow_all_pca_2048.csv', 'submissions/PLSR_image_1000_des_BOW_all_count_1_0.341.csv']

In [218]:
submission_weight = [1.0, 1.0, 1.0]

In [219]:
def calc_single(pred_list):
    score_dict = {}
    for i in range(len(pred_list)):
        for j in range(len(pred_list[i])):
            if pred_list[i][j] not in score_dict:
                score_dict[pred_list[i][j]] = 0.0
            score_dict[pred_list[i][j]] += float(len(pred_list[i]) - j) / len(pred_list[i]) * submission_weight[i]
    sorted_score = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)
    return map(lambda x: x[0], sorted_score[:20])

In [220]:
def ensemble():
    res = []
    dfs = [pd.read_csv(file_path, encoding='utf-8') for file_path in submission_file_list]
    for i in range(dfs[0].shape[0]):
        pred_list = [df.iloc[i]['Top_20_Image_IDs'].split(' ') for df in dfs]
        res.append(' '.join(calc_single(pred_list)))
    print len(res)
    return pd.DataFrame({'Descritpion_ID': dfs[0]['Descritpion_ID'].tolist(), 'Top_20_Image_IDs': res})

In [221]:
df_ensembled = ensemble()

2000


In [222]:
print df_ensembled

     Descritpion_ID                                   Top_20_Image_IDs
0             0.txt  1105.jpg 1480.jpg 292.jpg 105.jpg 1344.jpg 188...
1             1.txt  833.jpg 80.jpg 818.jpg 314.jpg 1820.jpg 457.jp...
2             2.txt  838.jpg 1724.jpg 159.jpg 1471.jpg 1183.jpg 136...
3             3.txt  1514.jpg 1778.jpg 26.jpg 469.jpg 1765.jpg 1215...
4             4.txt  922.jpg 742.jpg 985.jpg 1462.jpg 687.jpg 292.j...
5             5.txt  771.jpg 51.jpg 492.jpg 814.jpg 1874.jpg 846.jp...
6             6.txt  1855.jpg 1660.jpg 1815.jpg 1216.jpg 429.jpg 17...
7             7.txt  689.jpg 168.jpg 1285.jpg 409.jpg 627.jpg 975.j...
8             8.txt  419.jpg 531.jpg 719.jpg 1283.jpg 103.jpg 1847....
9             9.txt  549.jpg 570.jpg 1886.jpg 770.jpg 614.jpg 899.j...
10           10.txt  634.jpg 979.jpg 1366.jpg 42.jpg 1482.jpg 696.j...
11           11.txt  492.jpg 814.jpg 771.jpg 51.jpg 1943.jpg 1693.j...
12           12.txt  1653.jpg 845.jpg 1804.jpg 1656.jpg 676.jpg 190...
13    

In [223]:
df_ensembled.to_csv('submissions/ensembled_0.35_0.34_0.26_models_average.csv', index=False)